In [1]:
from scipy import stats
srcc = stats.spearmanr( [5,-1,8,2], [6,9,1,3])
srcc[0]

-0.7999999999999999

In [2]:
%cd ..


e:\books\uni\arshad\OTHER\ترم 3\پایاننامه\proposal\knowledge distillation\CKDN_code


In [3]:
import predict_one_image



In [4]:
iqa = predict_one_image.IQA_CKDN()

In [5]:
iqa.predict(restored_addr="images1/restored/3.png" , degraded_addr= "images1/degraded/3.png")  # dist , ref

c:\Users\Farhad\anaconda3\envs\gputest\lib\site-packages\torchvision\transforms\functional.py:423: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


0.3415382

In [6]:
dte = iqa.get_DTE_features("images1/restored/3.png")
qse = iqa.get_QSE_features("images1/restored/3.png")
qse_minus = iqa.get_QSE_minus_DTE_features("images1/restored/3.png" , "images1/restored/3.png" )

In [7]:
dte.shape , qse.shape , qse_minus.shape

((1, 256), (1, 256), (1, 256))

## correlation or cosine distance

In [8]:
from scipy import stats
import numpy as np
from scipy.spatial.distance import cosine


r = "S5/test/r.BMP"
d = "S5/test/i04_10_1.bmp"


rr = iqa.get_DTE_features(r)[0]
dd = iqa.get_DTE_features(d)[0]

srcc = stats.spearmanr( rr , dd)
srcc[0] , cosine(rr, dd)

(0.999831912336919, 2.3543834686279297e-05)

In [9]:
import glob
TID_ADDR = "E:\\books/uni/arshad/OTHER/term2\seminars\papers\Mengyang2019/bosse2017 varianced_patches/test_varianced_and_random_patch_method\datasets\TID2013"
dsts = ["01","02","03","04","05","06","07","08","09" , "10", "11", "12", "13", "14", "15", "16", "17", "18", "19","20","21","22","23","24"]


def get_images_dst(dst_type):
    imgs = glob.glob(TID_ADDR+'/distorted_images/*_'+ dst_type +'_'+ '*' +'.*')
    return imgs

def get_mos_dst(dst_type ):
    mos = []
    f = open(TID_ADDR+"/mos_with_names.txt", "r")
    lines = f.readlines()
    for l in lines:
        if l.split(' ')[1].split('_')[1] == dst_type:
            mos.append(float(l.split(' ')[0][:5]))

    return mos


refs = glob.glob(TID_ADDR+'/reference_images/*')

refs_for_dst = []

for elem in refs:
    for i in range(5):
        refs_for_dst.append(elem)

In [10]:
len(get_images_dst(dsts[7]))

125

In [11]:
len(get_images_dst(dsts[7])) , len(refs_for_dst) , len(get_mos_dst(dsts[7]))

(125, 125, 125)

In [12]:
get_images_dst(dsts[7])[107]

'E:\\books/uni/arshad/OTHER/term2\\seminars\\papers\\Mengyang2019/bosse2017 varianced_patches/test_varianced_and_random_patch_method\\datasets\\TID2013/distorted_images\\i22_08_3.bmp'

In [13]:
refs_for_dst[107]

'E:\\books/uni/arshad/OTHER/term2\\seminars\\papers\\Mengyang2019/bosse2017 varianced_patches/test_varianced_and_random_patch_method\\datasets\\TID2013/reference_images\\I22.BMP'

In [14]:
get_mos_dst(dsts[7])[107]

3.783

## on contents

In [25]:
contents = ["01","02","03","04","05","06","07","08","09" , "10", "11", "12", "13", "14", "15", "16", "17", "18", "19","20","21","22","23","24","25"]


def get_images_content(im_name):
    imgs = glob.glob(TID_ADDR+'/distorted_images/*'+im_name+'_'+ '*' +'_'+ '*' +'.*')
    return imgs

def get_mos_content( ):
    mos = []
    f = open(TID_ADDR+"/mos_with_names.txt", "r")
    lines = f.readlines()
    for l in lines:
        # if l.split(' ')[1][1:3] == im_name:
            mos.append(float(l.split(' ')[0][:5]))

    return mos


def get_refs_content(content):
        return glob.glob(TID_ADDR+'/reference_images/*'+content+'*') * 120

regression on type and level:

In [35]:
all = glob.glob(TID_ADDR+'/distorted_images/*')


all_types = []
all_levels = []

for im in all:
    type = im.split("\\")[-1][4:6]
    level = im.split("\\")[-1].split("_")[-1][0]
    all_types.append([float(type)])
    all_levels.append([float(level)])




all_levels = np.array(all_levels)
all_types = np.array(all_types)
features = np.hstack((all_types , all_levels))

moses = get_mos_content()

In [51]:
all_types

array([[ 1.],
       [ 1.],
       [ 1.],
       ...,
       [24.],
       [24.],
       [24.]])

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(features, moses, test_size=0.2, random_state=4342 , shuffle=True)

svr = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr.fit(X_train, y_train)

train_preds = svr.predict(X_train)
val_preds = svr.predict(X_val)

train_error = mean_squared_error(y_train, train_preds)
val_error = mean_squared_error(y_val, val_preds)

train_srcc = stats.spearmanr(y_train, train_preds)[0]
val_srcc = stats.spearmanr(y_val, val_preds)[0]

print(f'Train Error: {train_error}, Validation Error: {val_error}')
print(f'Train SRCC: {train_srcc}, Validation SRCC: {val_srcc}')


Train Error: 0.34588461227069456, Validation Error: 0.40633085904838545
Train SRCC: 0.8698893768381099, Validation SRCC: 0.8441641281625788
